In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from fbprophet import Prophet

In [2]:
weather = pd.read_csv('Jan_23/HistoricalWeather.csv')
weather.columns = ['Date', 'Region', 'Parameter', 'count', 'min', 'max', 'mean', 'variance']
weather['Date'] = pd.to_datetime(weather['Date'])

In [3]:
weather

,Date,Region,Parameter,count,min,max,mean,variance
0,2005-01-01,NSW,Precipitation,8.002343e+05,0.000000,1.836935,0.044274,0.028362
1,2005-01-01,NSW,RelativeHumidity,8.002343e+05,13.877194,80.522964,36.355567,253.559937
2,2005-01-01,NSW,SoilWaterContent,8.002343e+05,0.002245,0.414305,0.170931,0.007758
3,2005-01-01,NSW,SolarRadiation,8.002343e+05,14.515009,32.169781,26.749389,6.078587
4,2005-01-01,NSW,Temperature,8.002343e+05,14.485785,35.878704,27.341182,18.562212
...,...,...,...,...,...,...,...,...
245968,2021-01-18,WA,RelativeHumidity,2.528546e+06,13.516124,89.552414,31.117912,241.147935
245969,2021-01-18,WA,SoilWaterContent,2.528546e+06,0.000000,0.338951,0.054056,0.001309
245970,2021-01-18,WA,SolarRadiation,2.528546e+06,8.684816,32.555115,26.931745,24.668831
245971,2021-01-18,WA,Temperature,2.528546e+06,18.429735,36.473137,29.404902,16.040947


In [4]:
wildfires = pd.read_csv('Jan_23/Historical_Wildfires.csv')
wildfires

,Region,Date,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Mean_confidence,Std_confidence,Var_confidence,Count,Replaced
0,NSW,1/4/2005,8.680000,312.266667,42.400000,78.666667,2.886751,8.333333,3,R
1,NSW,1/5/2005,16.611250,322.475000,62.362500,85.500000,8.088793,65.428571,8,R
2,NSW,1/6/2005,5.520000,325.266667,38.400000,78.333333,3.214550,10.333333,3,R
3,NSW,1/7/2005,6.264000,313.870000,33.800000,92.200000,7.529940,56.700000,5,R
4,NSW,1/8/2005,5.400000,337.383333,122.533333,91.000000,7.937254,63.000000,3,R
...,...,...,...,...,...,...,...,...,...,...
26714,WA,1/18/2021,30.800000,330.909091,113.454545,86.636364,7.619353,58.054545,11,N
26715,WA,1/19/2021,2.000000,305.950000,15.800000,98.500000,2.121320,4.500000,2,N
26716,WA,1/20/2021,6.720000,335.137500,232.325000,94.250000,8.500000,72.250000,4,N
26717,WA,1/21/2021,198.362182,326.340909,230.754546,93.363636,6.325620,40.013468,55,N


In [6]:
vegetation = pd.read_csv('Jan_23/VegetationIndex.csv')
vegetation

,Region,Date,Vegetation_index_mean,Vegetation_index_max,Vegetation_index_min,Vegetation_index_std,Vegetation_index_variance
0,NSW,1/1/2005,0.349202,0.9972,-0.2,0.204862,0.041968
1,NSW,2/1/2005,0.357403,0.9772,-0.2,0.208673,0.043544
2,NSW,3/1/2005,0.354087,0.9750,-0.2,0.209450,0.043869
3,NSW,4/1/2005,0.347242,0.9904,-0.2,0.207307,0.042976
4,NSW,5/1/2005,0.345526,0.9972,-0.2,0.202858,0.041151
...,...,...,...,...,...,...,...
1339,WA,8/1/2020,0.255785,0.9692,-0.2,0.155347,0.024133
1340,WA,9/1/2020,0.234510,0.9849,-0.2,0.126898,0.016103
1341,WA,10/1/2020,0.213640,0.9782,-0.2,0.099860,0.009972
1342,WA,11/1/2020,0.205688,0.9919,-0.2,0.092952,0.008640


In [7]:
def region_dataframe(region):
    """
    returns dataframe with the following related to specific region:
    estimated_fire_area
    mean_Precipitation
    mean_RelativeHumidity
    mean_SoilWaterContent
    mean_SolarRadiation
    mean_Temperature
    mean_Windspeed
    vegetation_index_mean
    """
    
    wildfires = pd.read_csv('Jan_23/Historical_Wildfires.csv')
    wildfires['Date'] = pd.to_datetime(wildfires['Date'])
    wildfires = pd.pivot(wildfires, index = 'Date', columns = 'Region', values = ['Estimated_fire_area', 'Count'])
    columns = []
    for i,x in wildfires.columns:
        columns.append('{}_{}'.format(i,x))
    wildfires.columns = columns
    
    
    # Instantiating another dataframe with a daterange and merge to find the missing date.
    dummy = pd.DataFrame(index = pd.date_range(start = '2005-01-01', end = '2021-01-22'))


    # 2020-03-06 have missing values for all 7 states.
    wildfires = pd.merge(wildfires, dummy, how = 'outer', left_index= True, right_index=True)
    wildfires.fillna(0, inplace=True)
    print(wildfires.shape)

    df = wildfires[['Estimated_fire_area_{}'.format(region), 'Count_{}'.format(region)]]
    
    
    weather = pd.read_csv('Jan_23/HistoricalWeather.csv')
    weather.columns = ['Date', 'Region', 'Parameter', 'count', 'min', 'max', 'mean', 'variance']
    weather['Date'] = pd.to_datetime(weather['Date'])
    weather = weather.loc[weather['Region'].eq(region)]
    weather_pivot = pd.pivot(weather, index=['Date'], columns = ['Parameter', 'Region'], values = 'mean')

    columns = []
    for i,x in weather_pivot.columns:
        columns.append('mean_{}_{}'.format(i,x))
    weather_pivot.columns = columns
    weather_pivot.fillna(method = 'ffill', inplace=True)
    
    
    
    
    # Here we have monthly data that needs to be changed to daily.
    vegetation = pd.read_csv('Jan_23/VegetationIndex.csv')
    vegetation['Date'] = pd.to_datetime(vegetation['Date'])
    vegetation = vegetation.loc[vegetation['Region'].eq(region)]

    vegetation_pivot = pd.pivot(vegetation, index='Date', columns = 'Region').resample('1D').mean()
    vegetation_pivot = vegetation_pivot[['Vegetation_index_mean']]
    vegetation_pivot = vegetation_pivot[:'2020-12-01'].interpolate()
    vegetation_pivot = vegetation_pivot.unstack().reset_index()
    vegetation_pivot.columns = ['param', 'region', 'ds', 'y']
    vegetation_pivot = vegetation_pivot[['ds', 'y']]
    
    
    m = Prophet(
    changepoint_prior_scale= 30,
    holidays_prior_scale = 20,
    seasonality_prior_scale = 35,
    n_changepoints = 100,
    seasonality_mode = 'additive',
    daily_seasonality = False,
    weekly_seasonality=False,
    yearly_seasonality = False
        ).add_seasonality(name = 'monthly', period = 30.5, fourier_order=12
        ).add_seasonality(name = 'weekly', period = 7, fourier_order = 20
        ).add_seasonality(name = 'yearly', period = 365.25, fourier_order = 20
        ).add_seasonality(name = 'quarterly', period = 365.25/4, fourier_order = 5, prior_scale=15)
    
    print('fitting vegetation for {}'.format(region))
    m.fit(vegetation_pivot)
    future = m.make_future_dataframe(periods=52)
    pred = m.predict(future)
    vegetation_pivot = pred[['ds', 'yhat']]
    vegetation_pivot.rename(columns = {'ds':'Date', 'yhat':'Vegetation_index_mean_{}'.format(region)}, inplace= True)
    vegetation_pivot.set_index('Date', inplace=True)
    
#     for i in vegetation_pivot.columns:
#         vegetation_pivot[i] = vegetation_pivot[i].interpolate()
#     dummy = pd.DataFrame(index = pd.date_range(start = '2020-12-02', end = '2021-01-08'))
#     vegetation_pivot = pd.concat([vegetation_pivot, dummy], axis = 0)
#     for i in vegetation_pivot.columns:
#         vegetation_pivot[i] = vegetation_pivot[i].fillna(method='ffill')

        
        
#     columns = []
#     for i,x in vegetation_pivot.columns:
#         columns.append('{}_{}'.format(i,x))
#     vegetation_pivot.columns = columns

    df = pd.concat([df, weather_pivot, vegetation_pivot], axis = 1)

    df.reset_index(inplace=True)
    df.rename(columns ={'index':'Date'}, inplace=True)
    
    return df


In [10]:
regions = ['NSW', 'NT', 'QL', 'SA', 'TA', 'VI', 'WA']

In [11]:
for i in regions:
    region_dataframe(i).to_csv('{}_iso.csv'.format(i), index = False, header= True)

(5866, 14)
fitting vegetation for NSW


C:\Users\albert\Anaconda3\lib\site-packages\pandas\core\frame.py:4446: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(5866, 14)
fitting vegetation for NT


C:\Users\albert\Anaconda3\lib\site-packages\pandas\core\frame.py:4446: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(5866, 14)
fitting vegetation for QL


...

C:\Users\albert\Anaconda3\lib\site-packages\pandas\core\frame.py:4446: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(5866, 14)
fitting vegetation for VI


C:\Users\albert\Anaconda3\lib\site-packages\pandas\core\frame.py:4446: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



(5866, 14)
fitting vegetation for WA


C:\Users\albert\Anaconda3\lib\site-packages\pandas\core\frame.py:4446: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

